In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [11]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.000266                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000009
0.000750      2.0 -0.000255                  2.0 -0.000035
0.001052      3.0 -0.000230                  3.0  0.000010
0.001476      4.0 -0.000197                  4.0  0.000025
0.002070      5.0 -0.000155                  5.0  0.000043
0.002904      6.0 -0.000104                  6.0  0.000064
0.004074      7.0 -0.000041                  7.0  0.000089
0.005714      8.0  0.000035                  8.0  0.000119
0.008015      9.0  0.000127                  9.0  0.000154
0.011243     10.0  0.000236                 10.0  0.000196
0.015771     11.0  0.000366                 11.0  0.000247
0.022122     12.0  0.000519                 12.0  0.000306
0.031031     13.0  0.000697                 13.0  0.000378
0.043528     14.0  0.000878                 14.0  0.000451
0.061057     15.0  0.001032                 15.0  0.000514
0.085645     16.0  0.001173                 16.0  0.000575
0.120136     17.0  0.001323                 17.0  0.000642
0.168516     18.0  0.001482                 18.0  0.000716
0.236378     19.0  0.001652                 19.0  0.000797
0.331549     20.0  0.001835                 20.0  0.000886
0.465100     21.0  0.002015                 21.0  0.000976
0.652400     22.0  0.001826                 22.0  0.000909
0.915100     23.0  0.001248                 23.0  0.000671
1.283650     24.0  0.000743                 24.0  0.000454
1.800600     25.0  0.000412                 25.0  0.000299
2.525700     26.0  0.000205                 26.0  0.000191
3.542800     27.0  0.000081                 27.0  0.000116
4.969550     28.0  0.000023                 28.0  0.000070
6.970850     29.0  0.000007                 29.0  0.000045
9.778100     30.0  0.000002                 30.0  0.000029
13.715850    31.0 -0.000003                 31.0  0.000016
19.239350    32.0 -0.000007                 32.0  0.000006
26.987250    33.0 -0.000005                 33.0  0.000002
37.855300    34.0  0.000002                 34.0  0.000004
53.100050    35.0  0.000008                 35.0  0.000007
73.887500    36.0  0.000013                 36.0  0.000009
97.662500    37.0  0.000016                 37.0  0.000010
121.437500   38.0  0.000014                 38.0  0.000008
145.212500   39.0  0.000013                 39.0  0.000003
168.987500   40.0  0.000008                 40.0 -0.000007
192.762500   41.0 -0.000003                 41.0 -0.000023
216.537500   42.0 -0.000020                 42.0 -0.000046
240.312500   43.0 -0.000043                 43.0 -0.000073
264.087500   44.0 -0.000076                 44.0 -0.000108
287.862500   45.0 -0.000124                 45.0 -0.000155
311.637500   46.0 -0.000157                 46.0 -0.000189
335.412500   47.0 -0.000140                 47.0 -0.000181
359.187500   48.0 -0.000083                 48.0 -0.000138
382.962500   49.0 -0.000014                 49.0 -0.000108
406.737500   50.0  0.000093                 50.0 -0.000052
430.512500   51.0  0.000241                 51.0  0.000079
454.287500   52.0  0.000439                 52.0  0.000291
478.062500   53.0  0.000694                 53.0  0.000542
501.837500   54.0  0.001012                 54.0  0.000774
525.612500   55.0  0.001396                 55.0  0.000981
549.387500   56.0  0.001842                 56.0  0.001225
573.162500   57.0  0.002345                 57.0  0.001592
596.937500   58.0  0.002889                 58.0  0.002131
620.712500   59.0  0.003455                 59.0  0.002821
644.487500   60.0  0.004022                 60.0  0.003628
668.262500   61.0  0.004567                 61.0  0.004456
692.037500   62.0  0.005080                 62.0  0.005281
715.812500   63.0  0.005556                 63.0  0.006033
739.587500   64.0  0.006008                 64.0  0.006708
76

# Fluxes by Layer

CRD                            CLIRAD            \
                        flug          fldg     fnetg      flug      fldg   
pressure     level                                                         
0.000000e+00 1     -1.357878  0.000000e+00 -1.357878       NaN       NaN   
1.000000e-08 1           NaN           NaN       NaN -1.356134  0.000002   
6.244000e-04 2     -1.357878  6.161801e-09 -1.357878 -1.356134  0.000002   
8.759000e-04 3     -1.357878  8.887152e-09 -1.357878 -1.356134  0.000002   
1.228600e-03 4     -1.357878  1.349677e-08 -1.357878 -1.356134  0.000002   
1.723400e-03 5     -1.357878  2.129585e-08 -1.357878 -1.356134  0.000002   
2.417400e-03 6     -1.357878  3.445707e-08 -1.357878 -1.356134  0.000002   
3.390900e-03 7     -1.357878  5.661684e-08 -1.357878 -1.356134  0.000002   
4.756500e-03 8     -1.357878  9.382443e-08 -1.357878 -1.356134  0.000002   
6.672000e-03 9     -1.357878  1.561333e-07 -1.357878 -1.356134  0.000002   
9.358900e-03 10    -1.357878  2.602051e-07 -1.357878 -1.356134  0.000002   
1.312780e-02 11    -1.357878  4.335007e-07 -1.357878 -1.356134  0.000003   
1.841450e-02 12    -1.357878  7.211569e-07 -1.357878 -1.356134  0.000003   
2.583020e-02 13    -1.357878  1.197030e-06 -1.357877 -1.356134  0.000003   
3.623230e-02 14    -1.357878  1.980859e-06 -1.357876 -1.356133  0.000003   
5.082340e-02 15    -1.357878  3.242288e-06 -1.357875 -1.356133  0.000004   
7.129060e-02 16    -1.357878  5.197564e-06 -1.357872 -1.356132  0.000004   
1.000000e-01 17    -1.357877  8.169870e-06 -1.357868 -1.356132  0.000005   
1.402710e-01 18    -1.357875  1.266502e-05 -1.357862 -1.356130  0.000007   
1.967600e-01 19    -1.357872  1.942425e-05 -1.357852 -1.356128  0.000010   
2.759970e-01 20    -1.357866  2.951759e-05 -1.357837 -1.356125  0.000014   
3.871000e-01 21    -1.357857  4.445746e-05 -1.357813 -1.356119  0.000020   
5.431000e-01 22    -1.357842  6.615691e-05 -1.357775 -1.356111  0.000030   
7.617000e-01 23    -1.357820  9.231771e-05 -1.357728 -1.356100  0.000042   
1.068500e+00 24    -1.357799  1.160982e-04 -1.357683 -1.356089  0.000056   
1.498800e+00 25    -1.357779  1.343731e-04 -1.357645 -1.356079  0.000069   
2.102400e+00 26    -1.357762  1.466572e-04 -1.357615 -1.356070  0.000081   
2.949000e+00 27    -1.357748  1.529627e-04 -1.357595 -1.356063  0.000094   
4.136600e+00 28    -1.357737  1.538913e-04 -1.357583 -1.356059  0.000105   
5.802500e+00 29    -1.357730  1.516113e-04 -1.357579 -1.356056  0.000117   
8.139200e+00 30    -1.357726  1.491152e-04 -1.357577 -1.356057  0.000129   
1.141700e+01 31    -1.357724  1.480483e-04 -1.357576 -1.356060  0.000144   
1.601470e+01 32    -1.357727  1.492916e-04 -1.357578 -1.356068  0.000160   
2.246400e+01 33    -1.357738  1.545420e-04 -1.357583 -1.356083  0.000180   
3.151050e+01 34    -1.357758  1.696833e-04 -1.357589 -1.356107  0.000207   
4.420010e+01 35    -1.357790  2.043202e-04 -1.357586 -1.356141  0.000247   
6.200000e+01 36    -1.357836  2.671836e-04 -1.357569 -1.356190  0.000310   
8.577500e+01 37    -1.357898  3.650528e-04 -1.357533 -1.356256  0.000402   
1.095500e+02 38    -1.357961  4.717843e-04 -1.357489 -1.356324  0.000498   
1.333250e+02 39    -1.358029  5.802641e-04 -1.357449 -1.356399  0.000594   
1.571000e+02 40    -1.358136  7.246989e-04 -1.357411 -1.356513  0.000718   
1.808750e+02 41    -1.358295  9.054795e-04 -1.357389 -1.356680  0.000866   
2.046500e+02 42    -1.358522  1.124544e-03 -1.357398 -1.356915  0.001035   
2.284250e+02 43    -1.358821  1.367721e-03 -1.357454 -1.357219  0.001210   
2.522000e+02 44    -1.359207  1.632667e-03 -1.357575 -1.357605  0.001393   
2.759750e+02 45    -1.359712  1.923355e-03 -1.357789 -1.358104  0.001587   
2.997500e+02 46    -1.360388  2.250558e-03 -1.358137 -1.358761  0.001807   
3.235250e+02 47    -1.361261  2.681899e-03 -1.358579 -1.359599  0.002113   
3.473000e+02 48    -1.362310  3.335431e-03 -1.358974 -1.360605  0.002609   
3.710750e+02 49    -1.363501  4.294294e-03 -1.359207 -1.361754  0.003368 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')